In [34]:
from sqlalchemy import MetaData
from sqlalchemy import Table, Column
from sqlalchemy import Integer, String
from sqlalchemy import select

from sqlalchemy import create_engine

## What is database metadata?
- Describes the structure of the database, i.e tables, columns, constraints, in terms of data structures in Python
- Serves as the basis for SQL generation and object relational mapping
- Can generate to a schema, i.e turned into DDL that is emitted to the database
- Can be generated from a schema, i.e database introspection is performed to generate Python structures that represent existing tables, constraints, etc. 
- Forms the basis for database migration tools like SQLAlchemy Alembic


In [28]:
metadata = MetaData()
user_table = Table(
    "user_account",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("username", String(50), nullable=False),
    Column("fullname", String(255)),
)

In [29]:
# Table provides a single point of information regarding
# the structure of a table in a schema.
user_table.name

'user_account'

In [30]:
# The .c. attribute of Table is an associative array
# of Column objects, keyed on name.
user_table.c.username

Column('username', String(length=50), table=<user_account>, nullable=False)

In [31]:
print(user_table.c.username.name,' - ', user_table.c.username.type)

username  -  VARCHAR(50)


In [32]:
# It's a bit like a Python dictionary but not totally.
print(user_table.c)

ImmutableColumnCollection(user_account.id, user_account.username, user_account.fullname)


In [33]:
# Table has other information available, such as the collection
# of columns which comprise the table's primary key.
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [27]:
# The Table object is at the core of the SQL expression
# system - this is a quick preview of that.
print(select(user_table))

SELECT user_account.id, user_account.username, user_account.fullname 
FROM user_account


In [36]:
# create in-memory-only database
engine = create_engine('sqlite://', future=True)

In [37]:
metadata = MetaData()
user_table = Table(
    "user_account",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("username", String(50), nullable=False),
    Column("fullname", String(255)),
)

# Table and MetaData objects can be used to generate a schema
# in a database; MetaData features the create_all() method.
with engine.begin() as conn:
    metadata.create_all(conn)

In [38]:
# Types are represented using objects such as String, Integer,
# DateTime.  These objects can be specified as "class keywords",
# or can be instantiated with arguments.

from sqlalchemy import String, Numeric, DateTime, Enum

fancy_table = Table(
    "fancy",
    metadata,
    Column("key", String(50), primary_key=True),
    Column("timestamp", DateTime),
    Column("amount", Numeric(10, 2)),
    Column("type", Enum("a", "b", "c")),
)

with engine.begin() as conn:
    fancy_table.create(conn)
    
# at this point, the two Table objects we've created are part of a collection
# in the MetaData object called .tables
print(metadata.tables.keys())
metadata.tables['fancy']

dict_keys(['user_account', 'fancy'])


Table('fancy', MetaData(), Column('key', String(length=50), table=<fancy>, primary_key=True, nullable=False), Column('timestamp', DateTime(), table=<fancy>), Column('amount', Numeric(precision=10, scale=2), table=<fancy>), Column('type', Enum('a', 'b', 'c'), table=<fancy>), schema=None)

In [39]:
# table metadata also allows for constraints and indexes.
# ForeignKey is used to link one column to a remote primary
# key.  Note we can omit the datatype for a ForeignKey column.

from sqlalchemy import ForeignKey

addresses_table = Table(
    "email_address",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("email_address", String(100), nullable=False),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
)

with engine.begin() as conn:
    addresses_table.create(conn)
    
print(metadata.tables.keys())
metadata.tables['email_address']

dict_keys(['user_account', 'fancy', 'email_address'])


Table('email_address', MetaData(), Column('id', Integer(), table=<email_address>, primary_key=True, nullable=False), Column('email_address', String(length=100), table=<email_address>, nullable=False), Column('user_id', Integer(), ForeignKey('user_account.id'), table=<email_address>, nullable=False), schema=None)

In [46]:
# ForeignKey is a shortcut for ForeignKeyConstraint,
# which should be used for composite references.

from sqlalchemy import ForeignKeyConstraint
from sqlalchemy import Text

story_table = Table(
    "story",
    metadata,
    Column("story_id", Integer, primary_key=True),
    Column("version_id", Integer, primary_key=True),
    Column("headline", String(100), nullable=False),
    Column("body", Text),
)

published_table = Table(
    "published",
    metadata,
    Column("pub_id", Integer, primary_key=True),
    Column("pub_timestamp", DateTime, nullable=False),
    Column("story_id", Integer),
    Column("version_id", Integer),
    ForeignKeyConstraint(
        ["story_id", "version_id"], ["story.story_id", "story.version_id"]
    ),
)

### slide:: p
# create_all() by default checks for tables existing already
with engine.begin() as conn:
    metadata.create_all(conn)

In [40]:
# 'reflection' refers to loading Table objects based on
# reading from an existing database.
metadata2 = MetaData()

with engine.connect() as conn:
    user_reflected = Table("user_account", metadata2, autoload_with=conn)
    
# the user_reflected object is now filled in with all the columns
# and constraints and is ready to use
print(user_reflected.c)
print(user_reflected.primary_key)
print(select(user_reflected))

ImmutableColumnCollection(user_account.id, user_account.username, user_account.fullname)
PrimaryKeyConstraint(Column('id', INTEGER(), table=<user_account>, primary_key=True, nullable=False))
SELECT user_account.id, user_account.username, user_account.fullname 
FROM user_account


In [41]:
# Information about a database at a more specific level is available
# using the Inspector object.

from sqlalchemy import inspect

# inspector will work with an engine or a conneciton.
# no plans to change that :)
inspector = inspect(engine)

In [42]:
# the inspector provides things like table names:
inspector.get_table_names()

['email_address', 'fancy', 'user_account']

In [43]:
### slide:: p
# column information
inspector.get_columns("email_address")

[{'name': 'id',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'email_address',
  'type': VARCHAR(length=100),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'user_id',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0}]

In [44]:
### slide:: p
# constraints
inspector.get_foreign_keys("email_address")

[{'name': None,
  'constrained_columns': ['user_id'],
  'referred_schema': None,
  'referred_table': 'user_account',
  'referred_columns': ['id'],
  'options': {}}]

In [47]:
# The MetaData object also includes a feature that will reflect all the
# tables in a particular schema at once.

metadata3 = MetaData()
with engine.connect() as conn:
    metadata3.reflect(conn)
    
# the Table objects are then in the metadata.tables collection
story, published = metadata3.tables['story'], metadata3.tables['published']
print(select(story).join(published))

SELECT story.story_id, story.version_id, story.headline, story.body 
FROM story JOIN published ON story.story_id = published.story_id AND story.version_id = published.version_id
